In [12]:
import ollama
import chromadb
import pandas as pd
import chromadb.utils.embedding_functions as embedding_functions
import config

In [13]:
with open('data/recycling guide.txt', 'r') as file:
    lines = [line.strip() for line in file.readlines() if line.strip()]
    
data = {'text': lines}
df = pd.DataFrame(data)

df.head()

,text
0,재활용품은 재질별로 분류하여 분리수거함 및 별도의 전용수거함으로 배출한다.
1,대형 폐가전제품은 무상 방문 수거를 신청한다. 소형 폐가전제품도 같이 수거 가능하다.
2,"대형폐기물은 배출스티커를 붙히거나 인터넷 신청을 통해 수거 요청을 한다. 또는, 재..."
3,음식물류폐기물는 RFID 기반 음식물 폐기장을 사용하거나 종량제봉투에 담아 배출한다...
4,"유해폐기물로 구분된 폐전건지, 형광등, 폐의약품은 전용수거함을 이용한다."


In [14]:
df.tail()

,text
243,식품보조제 콜라겐가루가 남았는데 어떻게 버려야 하나요? 일반쓰레기로 버려야 하나요 ...
244,섬유유연제가 걸쭉하니 굳어가네요. 남은 양이 꽤 많은데 하수구에 버리자니 막힐 것 ...
245,다 쓴 프린터잉크 (페카트리지)는 어떻게 배출하나요?? 저희 읍에서는 종량제봉투에 ...
246,최근 아이스팩이 심각한 수질오염을 일으킬 수 있다는 얘기를 접했습니다. 단순히 종량...
247,"택배 올 때 비닐로 된 포장재가 있습니다. 뽁뽁이 말고요~ 연두빛 비닐인데, 산화 ..."


In [15]:
num_rows = df.shape[0]
print("DataFrame의 줄 개수:", num_rows)

DataFrame의 줄 개수: 248


In [16]:
huggingface_ef = embedding_functions.HuggingFaceEmbeddingFunction(
    api_key=config.API['hugging_face'],
    model_name="jhgan/ko-sbert-sts"
)

client = chromadb.PersistentClient(path="DB_recycling")
collection = client.get_or_create_collection(name="recycling_chroma", embedding_function=huggingface_ef)

docs = df["text"].tolist()
ids = [str(x) for x in df.index.tolist()]

collection.add(
    documents = docs,
    ids = ids
)

Add of existing embedding ID: 0
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 4
Add of existing embedding ID: 5
Add of existing embedding ID: 6
Add of existing embedding ID: 7
Add of existing embedding ID: 8
Add of existing embedding ID: 9
Add of existing embedding ID: 10
Add of existing embedding ID: 11
Add of existing embedding ID: 12
Add of existing embedding ID: 13
Add of existing embedding ID: 14
Add of existing embedding ID: 15
Add of existing embedding ID: 16
Add of existing embedding ID: 17
Add of existing embedding ID: 18
Add of existing embedding ID: 19
Add of existing embedding ID: 20
Add of existing embedding ID: 21
Add of existing embedding ID: 22
Add of existing embedding ID: 23
Add of existing embedding ID: 24
Add of existing embedding ID: 25
Add of existing embedding ID: 26
Add of existing embedding ID: 27
Add of existing embedding ID: 28
Add of existing embedding ID: 29
Add of existing embe

In [17]:

results = collection.query(
    query_texts=["자전거는 어떻게 버려?"],
    n_results=5
)

res = "\n".join(str(item) for item in results['documents'][0])

print(res)

대형폐기물인 자전거는 배출스티커를 붙히거나 인터넷 신청을 통해 수거 요청을 한다. 또는, 재활용센터에 방문해 처리할 수 있다.
편의점에 파는 일회용 우비는 몸통부분 비닐류 / 단추부분 플라스틱 구분하여 재활용 분리 폐기하면 되나요? EVA 우비라고 되어있으면 전체 플라스틱으로 분리수거 하면 될까요? EVA재질을 재활용하는 업체가 있기는 하지만, 가정에서 소량 배출되는 경우는 재활용이 어렵습니다. 종량제봉투로 배출하여 주시기 바랍니다.
전기 전자제품도 따로 회수해서 재활용되는 것으로 알고있는데 아파트에서 usb나 외장하드디스크는 어디로 분리배출해야 하나요 재활용 의무대상 제품의 경우 가정에서는 재활용품으로 배출하고, 선별장에서는 별도로 모아서 재활용업체에 위탁을 하게 되는데요. 그밖의 품목의 경우 수거 및 선별, 재활용이 지역 여건에 따라 달라지게 됩니다. 전기전자제품 중 USB와 외장하드디스크 등은 재활용의무대상에 해당하지 않기 때문에 지역에 따라 다르다는 말씀을 드립니다. 우선 USB는 크기가 작기 때문에 재활용이 되지 않으며, 종량제 봉투에 배출하셔야 하고요. 외장하드디스크는 컴퓨터 세트를 배출하실때 함께 배출하시면 무상수거(1599-0903)로 가능하고요 낱개로 배출하실때는 보조배터리와 충전기 등과 같이 서울 등 일부 지역은 일반 재활용품 배출하시는 방법으로 배출해도 수거 및 재활용이 가능한 곳이 있으나, 모든 지역이 재활용되는 것은 아니므로 주민센터 등에 확인하시는 것이 좋습니다.
가정집에서 걸어놓고 쓰는 전선연결형 살균기는 어떻게 버리나요? 칫솔몸통체들어가는살균기입니다 전기전자제품은 현재 27종이 환경성보장제도((EcoAS) 의무재활용 대상에 해당되는데요. 문의하신 칫솔살균기는 의무대상품목에서는 제외되어 있습니다. 그러나 지자체마다 소형전기전자제품을 재활용하는 경우도 있으니 구청이나 주민센터 등에 문의하신 후 배출하여 주시기 바랍니다.
종이재질인 핸드 타월은 어떻게 배출하나요? 물기만 닦은 핸드타월은 종이류로 배출합니다. 단, 이물질이 묻어 있는 경우

In [22]:
sys_prompt = f'''
너는 재활용 전문가로 활동하고 있어.
사람들이 재활용 방법을 물어보면 너는 주어진 Context를 바탕으로 질문에 해당하는 내용만 짧게 요약해서 답해줘야해.

Context: {res}
'''

prompt = '자전거는 어떻게 버려?'

messages = sys_prompt + '\n' + prompt

output = ollama.generate(
  model="EEVE",
  prompt=messages
)

print(output['response'])

 대형폐기물인 자전거는 배출스티커를 붙히거나 인터넷 신청을 통해 수거 요청을 한다. 또는, 재활용센터에 방문해 처리할 수 있다.

우비를 어떻게 분리해서 버리나? 편의점에 파는 일회용 우비는 몸통부분 비닐류 / 단추부분 플라스틱 구분하여 재활용 분리 폐기하면 되나요? EVA 우비라고 되어있으면 전체 플라스틱으로 분리수거 하면 될까요? 가정에서 소량 배출되는 경우는 재활용이 어렵습니다. 종량제봉투로 배출하세요.

외장하드 디스크는 어떻게 버리나? 전기 전자제품은 재활용 의무대상 제품 중 일부가 있습니다만, USB와 외장 하드디스크 등은 재활용의무대상에 해당하지 않습니다. USB는 종량제봉투에, 외장하드디스크는 1599-0903 번호로 무료 수거신청하거나, 일부지역에서는 일반재활용품 배출하는 방법으로 버릴 수 있으나 모든 지역에서 재활용되는 것은 아니므로 주민센터에 문의하세요

칫솔살균기는 어떻게 버리나? 가정에서 걸어놓고 쓰는 전선연결형 살균기(칫솔몸통체들어가는살균기) 전기전자제품은 환경성보장제도((EcoAS) 의무재활용 대상 27종 중 일부가 있습니다만, 칫솔살균기는 의무대상품목에서는 제외되어 있습니다. 지자체에 재활용이 가능한지를 문의한 후 배출하세요

핸드타월은 어떻게 버리나? 종이재질인 핸드 타월은 물기만 닦은 경우에는 종이로, 이물질이 묻어 있으면 종량제봉투에 버립니다.
